In [ ]:
year = [2020, 2021, 2022]
month = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

#비어있는 데이터 프레임 만들기.
df = pd.DataFrame(columns = ['ID','Time','Distance','Count','Fault'])
df['Fault']=df['Fault'].fillna(0)
df

#모델만들기
model = LogisticRegression()

for y in year:
    for m in month:
        if y == 2022 and m > 5: 
            print("종료")
            break
        else:
            user = pd.read_csv(f"/user/maria_dev/BDP/data/data{y}/data{y}_user/{y}_{m}.csv", index_col=0)
            fault = pd.read_csv(f"/user/maria_dev/BDP/data/data{y}/data{y}_fault/{y}_{m}.csv", index_col=0)
        
        # 1. data 새로운 값
        data = pd.merge(user,fault,on="ID", how="left")
        data["Count_y"] = data["Count_y"].fillna(0)
        data["Count_y"] = data["Count_y"].astype(int)
        data.columns = ['ID','Time','Distance','Count','Fault']
        data['Fault'] = np.where(data['Fault']>0,1,0)
        
        # 2. tmp - merge 위한 임시 df
        tmp = pd.merge(df,data,how='outer',on='ID')
        tmp = tmp.fillna(0)
        
                # 3. 학습 위한 df
        df['ID'] = tmp['ID']
        df['Time'] = (tmp['Time_x'] + tmp['Time_y']).astype(int)
        df['Distance'] = (tmp['Distance_x'] + tmp['Distance_y']).astype(int)
        df['Count'] = (tmp['Count_x'] + tmp['Count_y']).astype(int)
        df['Fault'] = (tmp['Fault_x'] + tmp['Fault_y']).astype(int)
        
        #4. 데이터 학습을 위해 정규화 하기.
        scaler = StandardScaler()
        train_features = scaler.fit_transform(df[['Time','Distance','Count']])
        train_labels = df['Fault']
        model.fit(train_features,train_labels)
            
    
        # 5. reset Fault = 1, to zero
#         df.loc[df['Fault'] > 0,['Time','Distance', 'Count', 'Fault']] = 0  
        
        # 5-1. 데이터 드랍
        idx = df.loc[df['Fault'] > 0].index
        df.drop(idx, inplace=True)
        
#         print(df.head(10))
#         print(df.shape)
        print(f"{y}년 {m}월 종료")
    
#테스트 데이터
user = pd.read_csv(f"/user/maria_dev/BDP/data/data2022/data2022_user/2022_6.csv",index_col = 0)
fault = pd.read_csv(f"/user/maria_dev/BDP/data/data2022/data2022_fault/2022_6.csv",index_col = 0)
data = pd.merge(user,fault,on="ID", how="left")
data["Count_y"] = data["Count_y"].fillna(0)
data["Count_y"] = data["Count_y"].astype(int)
data.columns = ['ID','Time','Distance','Count','Fault']
data['Fault'] = np.where(data['Fault']>0,1,0)

data_test = data[['Time','Distance','Count']]
print(data_test.head())


data_test_scaled = scaler.transform(data_test)
print(model.predict(data_test_scaled))

print(model.predict_proba(data_test_scaled))